<p style="text-align: center;font-size: 40pt">Overview of lidars</p>

In [1]:
%matplotlib widget
#%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib import animation
from IPython.display import HTML
import numpy as np
import scipy.stats as stats
import math
from PIL import Image

%run ./scripts/helper_func.py
path = "{0}/common/scripts/style.py".format(get_root_path())
%run $path

# Introduction

Lidars (LIght Detection And Ranging) are one of the most common sensors one can find on autonomous vehicles.
The reason behind their popularity is that they are becoming more and more affordable and provide rich 3D information about their surroundings.
In this lesson, you will learn the basics of how lidars work and what are their limitations. 

# Basic principle

## In one dimension
First of all, what is a lidar?
Well, a 1D lidar would be a [laser rangefinder](https://en.wikipedia.org/wiki/Laser_rangefinder), you know, those small devices used to measure a distance.
They send a laser pulse on an object and measure how much time it takes for the pulse to bounce on it and come back.
Knowing this time and the speed at which the pulse travels (the speed of light), one can compute the distance between the lidar and the object.

<p style="text-align: center;">
<img src="images/1d_lidar.svg" width="50%">
</p>

## In two dimensions
If you take this laser and point it to a mirror rotating along the Z axis, you can _scan_ a line around the sensor, meaning you now have a 2D lidar.
The output of such a lidar is no longer a single distance (or point), but a set of points (i.e. a point cloud) in 2D.
The position of a point is computed using the time it takes for a laser pulse to bounce back to the lidar as well as the mirror angle at the moment at which the pulse was fired.
This will give you an angle and a distance (i.e. [polar coordinates](https://en.wikipedia.org/wiki/Polar_coordinate_system)), which can then be converted to [2D Cartesian coordinates](https://en.wikipedia.org/wiki/Cartesian_coordinate_system#Cartesian_coordinates_in_two_dimensions) (in the frame of the lidar).

<p style="text-align: center;">
<img src="images/2d_lidar.gif" width="15%">
</p>

## In three dimensions
Finally, if you take multiple lasers (or channels or beams) and point them to the good old rotating mirror, you can _scan_ multiple lines (at different heights) around the sensor, which means you now have a 3D lidar.
The following image shows the output of such a lidar.
If you look carefully at the layout of the points, you will be able to see the scan lines (or rings) of the different lasers around the lidar.
When computing the position of a point in 3D, the difference with 2D is that positions are computed using two angles: the vertical angle (i.e. channel angle) as well as the horizontal/azimuth angle (i.e. mirror angle).
Therefore, you will have two angles and one distance (i.e. [spherical coordinates](https://en.wikipedia.org/wiki/Spherical_coordinate_system)), which can then be converted to [3D Cartesian coordinates](https://en.wikipedia.org/wiki/Cartesian_coordinate_system#Cartesian_coordinates_in_three_dimensions) (in the frame of the lidar).

<p style="text-align: center">
<img src="images/3d_lidar.jpg" width="50%">
</p>

# Now, in real life

In real life though, everything is not so simple.
Mostly, things go bad because of the physics of light propagation.

## Laser pulses are not instantaneous
Laser pulses are not just a punctual burst of energy, as we might think.
When a pulse is emitted by a lidar, the intensity released in function of time looks a little like this:

In [2]:
if 'fig' in globals():
    plt.close(fig)

fig = plt.figure(figsize=(5,5))

#------------------------
ax1 = fig.add_subplot(111)
ax = ax1
ax.set_title(r"Laser pulse intensity")
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.set_xlabel('time [s]')
ax.set_ylabel('intensity')

mu = 0
variance = 20
sigma = math.sqrt(variance)
x = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
_ = ax.plot(x, stats.norm.pdf(x, mu, sigma))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

When the pulse is reflected back, the intensity recorded by the lidar looks similar.
Even though this intensity variation happens in a really short amount of time, we cannot consider it as an instantaneous peak, since we are dealing with things happening at the speed of light here.
This leads to the following problem: when the pulse bounces back to the lidar, how to determine exactly the moment at which it arrives?
Most lidars have a threshold and when the intensity goes above and back below it, they take the time in the middle and state that this is when the pulse came back.
However, in some cases, the intensity recorded by the lidar looks more like this:

In [3]:
if 'fig' in globals():
    plt.close(fig)

fig = plt.figure(figsize=(5,5))

#------------------------
ax1 = fig.add_subplot(111)
ax = ax1
ax.set_title(r"Ugly returned laser pulse intensity")
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.set_xlabel('time [s]')
ax.set_ylabel('intensity')

mu_1 = 0
variance_1 = 20
sigma_1 = math.sqrt(variance_1)

mu_2 = 12
variance_2 = 20
sigma_2 = math.sqrt(variance_2)

x = np.linspace(mu_1 - 3*sigma_1, mu_2 + 3*sigma_2, 100)
_ = ax.plot(x, 0.6*stats.norm.pdf(x, mu_1, sigma_1) + 0.4*stats.norm.pdf(x, mu_2, sigma_2))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

This kind of intensity reading can lead to a bad detection of the time at which the pulse came back to the lidar, thus leading to a bad measure of distance.

## Laser pulses are not perfectly reflected back to lidars
Most of the time, [diffuse reflection](https://en.wikipedia.org/wiki/Diffuse_reflection) takes place and laser pulses are reflected in all directions when they hit an object, meaning that only a portion of their energy comes back to the lidar.
This leads to returned pulse intensities looking like this:

In [4]:
if 'fig' in globals():
    plt.close(fig)

fig = plt.figure(figsize=(5,5))

#------------------------
ax1 = fig.add_subplot(111)
ax = ax1
ax.set_title(r"Emitted and returned laser pulse intensities")
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.set_xlabel('time [s]')
ax.set_ylabel('intensity')

mu = 0
variance = 20
sigma = math.sqrt(variance)
x = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
_ = ax.plot(x, stats.norm.pdf(x, mu, sigma))
_ = ax.annotate('emitted pulse', xy=(3, 0.07), xytext=(13, 0.08), arrowprops=dict(facecolor='black', shrink=0.05))

_ = ax.text(17, 0.01, '...', fontsize=25)

mu = 40
variance = 20
sigma = math.sqrt(variance)
x = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
_ = ax.plot(x, stats.norm.pdf(x, mu, sigma)*0.5)
_ = ax.annotate('returned pulse', xy=(38, 0.04), xytext=(10, 0.05), arrowprops=dict(facecolor='black', shrink=0.05))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In some cases, the intensity of the returned pulse is so low that it is below the intensity threshold of the lidar and nothing is measured.
The intensity of the returned pulse varies in function of the distance to the surface, the incidence angle between the laser and the surface as well as the reflective properties of the surface.
If the surface is too polished, a [specular reflection](https://en.wikipedia.org/wiki/Specular_reflection) can even occur and the whole pulse will get reflected in a different direction.
Finally, some surfaces absorb completely the pulses.
This is the case of black surfaces and of water, which absorbs infrared wavelengths.

## Virtual points?
Lorem ipsum.

## Laser beams are shaped like cones
Shocking right?
Even though we are tempted to consider lidar laser beams like zero-width lines, in fact, they are shaped like cones (with their tip located in the lidar).
When a laser beam hits a surface, it looks more like this:

In [5]:
if 'fig' in globals():
    plt.close(fig)

fig = plt.figure(figsize=(5,5))

#------------------------
ax1 = fig.add_subplot(111)
ax = ax1
ax.set_title(r"Laser beam hitting a surface (seen from above)")
ax.set_axis_off()
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)

_ = ax.add_patch(patches.Wedge((0.025, 0.025), 1.05, 35, 42, color='white', alpha=0.2))
_ = ax.add_patch(patches.Circle((0.025, 0.025), radius=0.02, color='white'))
img = Image.open("images/turtle.png")
_ = ax.imshow(img, extent=[0.82, 0.92, 0.6, 0.7])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In this case, when the laser pulse comes back to the lidar, there is no way of knowing exactly where the object was within the cone of the beam.
This means that the farther you are from the lidar, the less precision you have, because the beam width increases with distance.

## Shadow points
Shadow points are points that do not really exist.
They are the result of averaging portions of a pulse reflected by different surfaces.
Indeed, because laser beams have a width, one emitted pulse can cause multiple intensity peaks at different times in the returned pulse.
Here is an example of how it could happen:

In [6]:
%matplotlib inline

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(7, 7))
plt.close(fig)
ax.set_title("Object geometry causing two \n intensity peaks in the returned pulse")

ax.set_axis_off()
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)

_ = ax.add_patch(patches.Wedge((0.025, 0.025), 1.055, 35, 42, color='white', alpha=0.2))
_ = ax.add_patch(patches.Circle((0.025, 0.025), radius=0.02, color='white'))
_ = ax.add_patch(patches.Polygon((np.array([[0.75, 0.7], [0.8, 0.64], [0.848, 0.678], [0.9, 0.61], [0.96, 0.658], [0.86, 0.785]]))))
poly_1 = patches.Arc((0.025, 0.025), 0.05, 0.05, 0, theta1=35, theta2=38.5, linewidth=3)
ax.add_patch(poly_1)
poly_2 = patches.Arc((0.025, 0.025), 0.05, 0.05, 0, theta1=38.5, theta2=42, linewidth=3)
ax.add_patch(poly_2)

def draw(t):
    if t > 2.11:
        poly_1.center = (1.716, 1.287)
        poly_1.angle = 180
    poly_1.width = t
    poly_1.height = t
    if t > 2:
        poly_2.center = (1.551, 1.317)
        poly_2.angle = 180
    poly_2.width = t
    poly_2.height = t
    return (poly_1, poly_2)
HTML(animation.FuncAnimation(fig, func=draw, frames=np.arange(0.025, 4.22, 0.02), interval=40, blit=True, repeat=False).to_html5_video())

In this case, the returned pulse intensity would look something like this:

In [11]:
%matplotlib widget
if 'fig' in globals():
    plt.close(fig)

fig = plt.figure(figsize=(5,5))

#------------------------
ax1 = fig.add_subplot(111)
ax = ax1
ax.set_title(r"Returned pulse intensity")
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.set_xlabel('time [s]')
ax.set_ylabel('intensity')

mu_1 = 0
variance_1 = 20
sigma_1 = math.sqrt(variance_1)

mu_2 = 15
variance_2 = 20
sigma_2 = math.sqrt(variance_2)

x = np.linspace(mu_1 - 3*sigma_1, mu_2 + 3*sigma_2, 100)
_ = ax.plot(x, 0.5*stats.norm.pdf(x, mu_1, sigma_1) + 0.5*stats.norm.pdf(x, mu_2, sigma_2))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

If the intensity in the middle section of this plot is above the lidar intensity threshold, the distance computed for the point will be located between the two surfaces and its position will be:

In [12]:
if 'fig' in globals():
    plt.close(fig)

fig = plt.figure(figsize=(5,5))

#------------------------
ax1 = fig.add_subplot(111)
ax = ax1
ax.set_title(r"Shadow point location")
ax.set_axis_off()
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)

_ = ax.add_patch(patches.Wedge((0.025, 0.025), 1.055, 35, 42, color='white', alpha=0.2))
_ = ax.add_patch(patches.Circle((0.025, 0.025), radius=0.02, color='white'))
_ = ax.add_patch(patches.Polygon((np.array([[0.75, 0.7], [0.8, 0.64], [0.848, 0.678], [0.9, 0.61], [0.96, 0.658], [0.86, 0.785]])), alpha=1))
_ = ax.add_patch(patches.Circle((0.83, 0.66), radius=0.01, color='red'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

It results in a point being placed between two surfaces, where nothing was really measured.
This can happen when things are placed in front of each others in a scan.
Some lidars support multiple _echoes_ and will place two points at different distances instead.

## Laser incidence angle distorts the measurements
As surprising as this might sound, when the incidence angle between the laser beam and a surface is too high, the distance measured by the lidar is smaller than it should.
This is due to the fact that most of the laser pulse intensity is reflected back to the lidar when the first portion of the beam cone hits the surface, as illustrated here:

<p style="text-align: center">
<img src="images/bias.png" width="50%">
</p>

Here, the color represents the intensity distribution of the laser pulse.
In this case, because we are intersecting a cone and a plane, this distribution is shaped like an ellipse.
Most of the intensity of the laser pulse is concentrated on the side of the ellipse which is closer to the lidar, rather than in its center, like it would with a small incidence angle.
This results in a distance measurement smaller than reality, because most of the beam intensity is reflected back to the lidar before the cone center hits the surface.
Although this bias is mostly caused by the incidence angle between the laser beam and the surface it hits, it is also influenced by distance.
Also, between lidar models, differences exist because of the different laser beam aperture angles, electronic components, etc.

# Lidar specifications

- laser beam divergence -> (degrees)
- range/angle precision (influenced by errors and distance + beam divergence) -> (meters)
- range/angle accuracy (influenced by errors) -> (meters)

- horizontal field of view -> the horizontal area captured by the sensor (degrees)
- horizontal (or azimuth) angular resolution -> smallest horizontal angle between two successive points (degrees)
- horizontal points -> the number of horizontal points generated per scan
- horizontal points -> horizontal field of view / horizontal angular resolution

- vertical field of view -> the vertical area captured by the sensor (degrees)
- vertical angular resolution -> smallest vertical angle between two successive points (degrees)
- vertical points -> the number of rows of points generated per scan, also called channels or beams
- vertical points = vertical field of view / vertical angular resolution

- scan rate -> number of scans generated per second
- points per second -> total number of points generated in a second
- points per second = horizontal points * vertical points * scan rate

- laser class safety

- minimum range -> smallest distance from target from which the sensor can operate (meters)
- maximum range -> biggest distance from target from which the sensor can operate (meters)

interaction between metrics, but marketing people tend to hide it